# Libraries

In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# to save and load python data
import pickle
# numerical processing
import numpy as np

# Data

In [2]:
# load pickled data
# =================

# load data
cl_1 = pickle.load(open('crashes_list_1', 'rb'))
cl_2 = pickle.load(open('crashes_list_2', 'rb'))
cl_3 = pickle.load(open('crashes_list_3', 'rb'))
cl_4 = pickle.load(open('crashes_list_4', 'rb'))
cl_5 = pickle.load(open('crashes_list_5', 'rb'))

# combine lists
cl = cl_1 + cl_2 + cl_3 + cl_4 + cl_5

# total no. of items in the list
len(cl)

4983

# Cleaning

In [3]:
# dataframes
# ==========

# concatenate dataframes
c_df = pd.concat(cl)
# drop unwanted rows
c_df = c_df.drop('-', axis=1)
# reset at index
c_df = c_df.reset_index(drop=True)
# replace '?' with np.nan
c_df = c_df.replace('?', np.nan)
# rename columns
c_df.columns = [col_name[:-1] for col_name in c_df.columns]
c_df.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn / ln,Aboard,Fatalities,Ground,Summary
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2 (passengers:1 crew:1),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1 (passengers:0 crew:1),1 (passengers:0 crew:0),0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20 (passengers:? crew:?),14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encou...


### Extract aboard and fatalities count

In [4]:
# extract counts
# ==============

# extract values
c_df['Total Aboard'] = c_df['Aboard'].str.extract('(\d*)')
c_df['Passengers Aboard'] = c_df['Aboard'].str.extract('passengers:(\d*)')
c_df['Crew Aboard'] = c_df['Aboard'].str.extract('crew:(\d*)')

c_df['Total Fatalities'] = c_df['Fatalities'].str.extract('(\d*)')
c_df['Passengers Fatalities'] = c_df['Fatalities'].str.extract('passengers:(\d*)')
c_df['Crew Fatalities'] = c_df['Fatalities'].str.extract('crew:(\d*)')

# drop columns
c_df = c_df.drop(['Aboard', 'Fatalities'], axis=1)
# replace empty strins with np.nan
c_df = c_df.replace('', np.nan)

### Fix time format

In [5]:
# extract time from Time column
c_df['Time'] = c_df['Time'].str.extract('([0-9:]*)')

In [6]:
c_df['Time'].str.len().value_counts()

4.0    3246
5.0     194
0.0      40
3.0       3
2.0       1
Name: Time, dtype: int64

In [7]:
# fix time format
# ===============

# time with 0 characters
# no need to fix

# time with 1 characters
# none found

# time with 2 characters
# change add 00 minutes to hh ( hh -> hh:00)
c_df.loc[c_df['Time'].str.len()==2, 'Time'] = c_df.loc[c_df['Time'].str.len()==2, 'Time'] + ':00'

# time with 3 characters 
# change hmm to h:mm
c_df.loc[c_df['Time'].str.len()==3, 'Time'] = c_df.loc[c_df['Time'].str.len()==3, 'Time'].str[:1] + ':' + c_df.loc[c_df['Time'].str.len()==3, 'Time'].str[1:]

# time with 4 characters and has : character
# no need to change

# time with 4 characters and dont have : character
# change hhmm to hh:mm
c_df.loc[np.logical_and(c_df['Time'].str.len()==4, c_df['Time'].str.contains(':')==False), 'Time'] = c_df.loc[np.logical_and(c_df['Time'].str.len()==4, c_df['Time'].str.contains(':')==False), 'Time'].str[:2] + ':' + c_df.loc[np.logical_and(c_df['Time'].str.len()==4, c_df['Time'].str.contains(':')==False), 'Time'].str[2:]

# time with 5 characters and has : character
# already in proper format

# time with 5 characters and dont have : character
# none found

In [8]:
c_df['Time'].str.len().value_counts()

5.0    3436
0.0      40
4.0       8
Name: Time, dtype: int64

### Fix columns order

In [9]:
# list of columns
c_df.columns

Index(['Date', 'Time', 'Location', 'Operator', 'Flight #', 'Route', 'AC  Type',
       'Registration', 'cn / ln', 'Ground', 'Summary', 'Total Aboard',
       'Passengers Aboard', 'Crew Aboard', 'Total Fatalities',
       'Passengers Fatalities', 'Crew Fatalities'],
      dtype='object')

In [10]:
# reorder columns
c_df = c_df[['Date', 'Time', 'Location', 'Operator', 'Flight #', 
             'Route', 'AC  Type', 'Registration', 'cn / ln', 
             'Total Aboard', 'Passengers Aboard', 'Crew Aboard', 
             'Total Fatalities', 'Passengers Fatalities', 'Crew Fatalities', 
             'Ground', 'Summary']]

c_df.head()

,Date,Time,Location,Operator,Flight #,Route,AC Type,Registration,cn / ln,Total Aboard,Passengers Aboard,Crew Aboard,Total Fatalities,Passengers Fatalities,Crew Fatalities,Ground,Summary
0,"September 17, 1908",17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",NaN,"Juvisy-sur-Orge, France",NaN,NaN,Air show,Wright Byplane,SC1,NaN,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",06:30,"Atlantic City, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",NaN,"Victoria, British Columbia, Canada",Private,NaN,NaN,Curtiss seaplane,NaN,NaN,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20,NaN,NaN,14,NaN,NaN,0,The airship flew into a thunderstorm and encou...


# Save data

In [11]:
# save data in a csv file
c_df.to_csv('crashes.csv', index=False)